In [1]:
# Load packages
#%%
import numpy as np
import pandas as pd
import ast
from sklearn.preprocessing import MinMaxScaler
import torch
from main import *

In [12]:
#%% Sensitivity analysis - Baseline

# Real-world - train
df_all = pd.read_csv('data/food_df_rob.csv') 
year = 2018
df = df_all[df_all['year'] == year]

# Prepare real world data
X = df[['ad_exc2_aid_gdp', 'infl', 'food_prod', 'food_imp_net_gdp',
        'log_gdp', 'log_gdp_pc', 'gdp_pc_grw', 'gov_score', 'e_pt_coup', 'disast_affect', 'fight', 'prec_z', 'net_dens']].to_numpy()
A = df['ad_sdg2_aid'].to_numpy()
Y = df['change_msfi'].to_numpy()

n = X.shape[0]
p = X.shape[1]

# Data standardization: min-max scaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[:,2:])
data_train = np.concatenate([data[:,0:2], data_scaled], axis=1)

# Hyperpar list
hyper_opt_list = open("hyperpars/hyperpars_opt_real.txt", "r")
hyper_opt_list = hyper_opt_list.read()
hyper_opt = ast.literal_eval(hyper_opt_list)
opt_hyperpars = hyper_opt[-5]

# Set all seeds
np.random.seed(123)
torch.manual_seed(123)

# Estimate model
data_train_bae, mod_BAE = BAE(data_train, opt_hyperpars)    
device = torch.device('cpu')        
aid_max = np.max(data_train[:,1])
aid_min = np.min(data_train[:,1])
aid_random = np.random.uniform(aid_min, aid_max, (data_train_bae.shape[0], opt_hyperpars['m_scw']))
data_train_scw = SCw(data_train_bae, aid_random, opt_hyperpars)
_, _, a_base = GPS(data_train_scw)

In [ ]:
#%% Sensitivity analysis w.r.t. past aid of neighbor countries

# Real-world - train
df_all = pd.read_csv('data/food_df_rob.csv') 
year = 2018
df = df_all[df_all['year'] == year]

# Prepare real world data
X = df[['ad_exc2_aid_gdp', 'infl', 'food_prod', 'food_imp_net_gdp',
        'log_gdp', 'log_gdp_pc', 'gdp_pc_grw', 'gov_score', 'e_pt_coup', 'disast_affect', 'fight', 'prec_z', 'net_dens']].to_numpy()
A = df['ad_sdg2_aid'].to_numpy()
Y = df['change_msfi'].to_numpy()

n = X.shape[0]
p = X.shape[1] + 1

# Add covariate
borders = pd.read_csv('data/borders.csv')
borders = borders[borders['country_code'].isin(df['country'])]
borders = borders[borders['country_border_code'].isin(df['country'])]
past_aid_neigh = np.empty((n,1))
for i in range(n):
    ctry = df['country'].values[i]
    neighs = borders[borders['country_code'] == ctry]['country_border_code'].values
    if len(neighs) > 0:
        past_aid_neigh[i] = np.mean(df_all[(df_all['year'] == 2017) & (df_all['country'].isin(neighs))]['ad_sdg2_aid'])
    else:
        past_aid_neigh[i] = df_all[(df_all['year'] == 2017) & (df_all['country'] == ctry)]['ad_sdg2_aid']
    
data = np.concatenate([Y.reshape(n,1), A.reshape(n,1), X, past_aid_neigh],axis=1)

# Data standardization: min-max scaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[:,2:])
data_train = np.concatenate([data[:,0:2], data_scaled], axis=1)

# Hyperpar list
hyper_opt_list = open("hyperpars/hyperpars_opt_real.txt", "r")
hyper_opt_list = hyper_opt_list.read()
hyper_opt = ast.literal_eval(hyper_opt_list)
opt_hyperpars = hyper_opt[-5]

# Set all seeds
np.random.seed(123)
torch.manual_seed(123)

# Estimate model
data_train_bae, mod_BAE = BAE(data_train, opt_hyperpars)    
device = torch.device('cpu')        
aid_max = np.max(data_train[:,1])
aid_min = np.min(data_train[:,1])
aid_random = np.random.uniform(aid_min, aid_max, (data_train_bae.shape[0], opt_hyperpars['m_scw']))
data_train_scw = SCw(data_train_bae, aid_random, opt_hyperpars)
_, _, a_neighaid = GPS(data_train_scw)

In [ ]:
#%% Sensitivity analysis w.r.t. past aid 

# Real-world - train
df_all = pd.read_csv('data/food_df_ana.csv') 
year = 2018
df = df_all[df_all['year'] == year]

# Prepare real world data
X = df.iloc[:, 5:].to_numpy()
A = df.iloc[:, 4].to_numpy()
Y = df.iloc[:, 2].to_numpy()

n = X.shape[0]
p = X.shape[1] + 1

# Add covariate
past_aid = df_all[df_all['year'] == 2017]['ad_sdg2_aid'].values.reshape(n,1)

data = np.concatenate([Y.reshape(n,1), A.reshape(n,1), X, past_aid],axis=1)

# Data standardization: min-max scaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[:,2:])
data_train = np.concatenate([data[:,0:2], data_scaled], axis=1)

# Hyperpar list
hyper_opt_list = open("hyperpars/hyperpars_opt_real.txt", "r")
hyper_opt_list = hyper_opt_list.read()
hyper_opt = ast.literal_eval(hyper_opt_list)
opt_hyperpars = hyper_opt[-5]

# Set all seeds
np.random.seed(123)
torch.manual_seed(123)

# Estimate model
data_train_bae, mod_BAE = BAE(data_train, opt_hyperpars)    
device = torch.device('cpu')        
aid_max = np.max(data_train[:,1])
aid_min = np.min(data_train[:,1])
aid_random = np.random.uniform(aid_min, aid_max, (data_train_bae.shape[0], opt_hyperpars['m_scw']))
data_train_scw = SCw(data_train_bae, aid_random, opt_hyperpars)
_, _, a_pastaid = GPS(data_train_scw)

In [ ]:
#%% Sensitivity analysis w.r.t. past Insecurity Prevalence

# Real-world - train
df_all = pd.read_csv('data/food_df_rob.csv') 
year = 2018
df = df_all[df_all['year'] == year]

# Prepare real world data
X = df[['ad_exc2_aid_gdp', 'infl', 'food_prod', 'food_imp_net_gdp',
        'log_gdp', 'log_gdp_pc', 'gdp_pc_grw', 'gov_score', 'e_pt_coup', 'disast_affect', 'fight', 'prec_z', 'net_dens']].to_numpy()
A = df['ad_sdg2_aid'].to_numpy()
Y = df['change_msfi'].to_numpy()

n = X.shape[0]
p = X.shape[1] + 1

# Add covariate
past_prev_msfi = df_all[df_all['year'] == 2017]['prev_msfi'].values.reshape(n,1)

data = np.concatenate([Y.reshape(n,1), A.reshape(n,1), X, past_prev_msfi],axis=1)

# Data standardization: min-max scaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[:,2:])
data_train = np.concatenate([data[:,0:2], data_scaled], axis=1)

# Hyperpar list
hyper_opt_list = open("hyperpars/hyperpars_opt_real.txt", "r")
hyper_opt_list = hyper_opt_list.read()
hyper_opt = ast.literal_eval(hyper_opt_list)
opt_hyperpars = hyper_opt[-5]

# Set all seeds
np.random.seed(123)
torch.manual_seed(123)

# Estimate model
data_train_bae, mod_BAE = BAE(data_train, opt_hyperpars)    
device = torch.device('cpu')        
aid_max = np.max(data_train[:,1])
aid_min = np.min(data_train[:,1])
aid_random = np.random.uniform(aid_min, aid_max, (data_train_bae.shape[0], opt_hyperpars['m_scw']))
data_train_scw = SCw(data_train_bae, aid_random, opt_hyperpars)
_, _, a_pastins = GPS(data_train_scw)

In [ ]:
#%% Sensitivity analysis w.r.t. past Insecurity Prevalence of neighbor countries 

# Real-world - train
df_all = pd.read_csv('data/food_df_rob.csv') 
year = 2018
df = df_all[df_all['year'] == year]

# Prepare real world data
X = df[['ad_exc2_aid_gdp', 'infl', 'food_prod', 'food_imp_net_gdp',
        'log_gdp', 'log_gdp_pc', 'gdp_pc_grw', 'gov_score', 'e_pt_coup', 'disast_affect', 'fight', 'prec_z', 'net_dens']].to_numpy()
A = df['ad_sdg2_aid'].to_numpy()
Y = df['change_msfi'].to_numpy()
n = X.shape[0]
p = X.shape[1] + 1

# Add covariate
borders = pd.read_csv('data/borders.csv')
borders = borders[borders['country_code'].isin(df['country'])]
borders = borders[borders['country_border_code'].isin(df['country'])]
past_msfe_neigh = np.empty((n,1))
for i in range(n):
    ctry = df['country'].values[i]
    neighs = borders[borders['country_code'] == ctry]['country_border_code'].values
    if len(neighs) > 0:
        past_hiv_neigh[i] = np.mean(df_all[(df_all['year'] == 2017) & (df_all['country'].isin(neighs))]['prev_msfi'])
    else:
        past_hiv_neigh[i] = df_all[(df_all['year'] == 2017) & (df_all['country'] == ctry)]['prev_msfi']
    
data = np.concatenate([Y.reshape(n,1), A.reshape(n,1), X, past_msfe_neigh],axis=1)

# Data standardization: min-max scaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[:,2:])
data_train = np.concatenate([data[:,0:2], data_scaled], axis=1)

# Hyperpar list
hyper_opt_list = open("hyperpars/hyperpars_opt_real.txt", "r")
hyper_opt_list = hyper_opt_list.read()
hyper_opt = ast.literal_eval(hyper_opt_list)
opt_hyperpars = hyper_opt[-1]

# Set all seeds
np.random.seed(123)
torch.manual_seed(123)

# Estimate model
data_train_bae, mod_BAE = BAE(data_train, opt_hyperpars)    
device = torch.device('cpu')        
aid_max = np.max(data_train[:,1])
aid_min = np.min(data_train[:,1])
aid_random = np.random.uniform(aid_min, aid_max, (data_train_bae.shape[0], opt_hyperpars['m_scw']))
data_train_scw = SCw(data_train_bae, aid_random, opt_hyperpars)
_, _, a_pastinsneigh = GPS(data_train_scw)

In [ ]:
a_base
a_pastaid
a_neighaid
a_pastinsneigh

array([[-8.03338767e-02],
       [ 1.80656374e-04],
       [-2.39857360e-07],
       [ 6.68677886e+01],
       [-1.35234097e+04],
       [-9.69251322e-02]])